# CDO bindings test

Before using the cdo bindigs we need to import the cdo bindings package (`python-cdo` in `conda` or `mamba`).
Additionally, before calling an operator, an object has to be created first.

We make use of the `tempdir` argument in order to store temporary data output in a custom folder and avoid heavy memory usage.

In [1]:
import cdo
import tools

tempPath = '/scratch/users/mnurisso/tempcdo/'
cdo = cdo.Cdo(tempdir=tempPath)

In [2]:
# Load some test data

path = '/work/datasets/obs/ERA5/mean_sea_level_pressure/mon/'
filename = 'ERA5_mean_sea_level_pressure_mon_0.25x0.25_sfc_1979-2021.nc'

infile = path+filename
print(infile)

/work/datasets/obs/ERA5/mean_sea_level_pressure/mon/ERA5_mean_sea_level_pressure_mon_0.25x0.25_sfc_1979-2021.nc


## Simple command test

We try to run a simple `showname` cdo command.

In [3]:
cdo.showname(input=infile)

['msl']

## NAO index cdo pipeline

We evaluate the NAO index in a similar way of what is the function `station_based_index()` in the `index.py` file.
The result of the analysis has to be compared with what is obtained with the `station_based_index()` function in order to test the function itself.

### Anomalies

In [4]:
msl_ma = cdo.monmean(input=infile)
msl_ma_av = cdo.timmean(input=msl_ma)
msl_an = cdo.sub(input=[msl_ma, msl_ma_av])

Commands can also be concatenated like `cdo.timmean(input= '-monmean' + infile)`

### Interpolation to NAO coordinates

In [5]:
# Get the latitude and longitudes for NAO

diagname  = 'teleconnections'
telecname = 'NAO'

namelist = tools.load_namelist(diagname)

lon1 = tools.lon_180_to_360(namelist[telecname]['lon1'])
lat1 = namelist[telecname]['lat1']
lon2 = tools.lon_180_to_360(namelist[telecname]['lon2'])
lat2 = namelist[telecname]['lat2']

# Interpolate to the acquired coordinates

field_an1 = cdo.remapnn("lon={0}_lat={1}".format(lon1,lat1),input=msl_an)
field_an2 = cdo.remapnn("lon={0}_lat={1}".format(lon2,lat2),input=msl_an)

### Evaluate NAO index

In [6]:
# Rolling average

months = 3
field_an1_ma = cdo.runmean("{0}".format(months),input=field_an1)
field_an2_ma = cdo.runmean("{0}".format(months),input=field_an2)

In [7]:
# NAO (not index)

diff_ma = cdo.sub(input=[field_an1_ma, field_an2_ma])
mean_ma = cdo.timmean(input=diff_ma)
std_ma = cdo.timstd(input=diff_ma)

In [8]:
# Index NAO
sub_nao = cdo.sub(input=[diff_ma,mean_ma])

# setup output
machine = 'wilma'
config = tools.load_config(machine)

ofile  = config[diagname]['output']
ofile += '/cdo_nao.nc'

cdo.div(input=[sub_nao,std_ma],output=ofile)

'/scratch/users/mnurisso/aqua/teleconnections/cdo_nao.nc'

## Cleanup the temporary files

Temporary files are not deleted when the jupyter notebook is closed. 
In order to empty the space we need to run an additional command.

In [9]:
cdo.cleanTempDir()